# Usuarios y títulos

In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Socios activos 2016

In [2]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv(
    'socios_activos_2016_ciudades_barrios.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 365 ms, sys: 11.9 ms, total: 377 ms
Wall time: 376 ms


In [3]:
perfiles_df.shape

(50272, 11)

In [4]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,soledad,bogota,CEM,,,,,


# Calculando valores frecuentes

In [5]:
groups_df = perfiles_df.copy()

In [6]:
groups_df.shape

(50272, 11)

#### Sucursal

In [7]:
groups_df['Sucursal'].nunique()

31

In [8]:
groups_df['Sucursal'].value_counts()

BLAA      24278
TUN        3165
IBA        2339
PAS        2089
POP        2078
NEI        1978
PER        1922
MAN        1159
GIR        1090
IPI        1041
CAR         758
SAN         728
CASAGO      686
VAL         660
FLO         611
ARM         575
HON         544
LET         526
BUC         511
QUI         478
STM         438
SIN         438
BUE         337
BAR         328
VIL         298
CAL         287
RIO         250
CUC         236
MED         219
MON         209
             16
Name: Sucursal, dtype: int64

In [9]:
%%time

ciudades_sucursales = {
    'armenia': 'ARM',
    'barranquilla': 'BAR',
    'bogota': 'BLAA',
    'bucaramanga': 'BUC',
    'buenaventura': 'BUE',
    'cali': 'CAL',
    'cartagena': 'CAR',
    'cucuta': 'CUC',
    'florencia': 'FLO',
    'girardot': 'GIR',
    'honda': 'HON',
    'ibague': 'IBA',
    'ipiales': 'IPI',
    'leticia': 'LET',
    'manizales': 'MAN',
    'medellin': 'MED',
    'monteria': 'MON',
    'neiva': 'NEI',
    'pasto': 'PAS',
    'pereira': 'PER',
    'popayan': 'POP',
    'quibdo': 'QUI',
    'riohacha': 'RIO',
    'san andres': 'SAN',
    'santa marta': 'STM',
    'sincelejo': 'SIN',
    'tunja': 'TUN',
    'valledupar': 'VAL',
    'villavicencio': 'VIL',
}

def fill_blank_sucursal(x):
    r = x['Sucursal']
    if not r:
        c = x['Ciudad']
        if c in ciudades_sucursales:
            r = ciudades_sucursales[c]
    return r

groups_df['Sucursal'] = groups_df.apply(fill_blank_sucursal, axis=1)

CPU times: user 1.48 s, sys: 0 ns, total: 1.48 s
Wall time: 1.48 s


In [10]:
%%time

cn = groups_df['Sucursal'].value_counts()

def group_sucursal(x):
    e = x['Sucursal']
    if e and cn[e] >= 0:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Sucursal'] = groups_df.apply(group_sucursal, axis=1)

CPU times: user 2.1 s, sys: 0 ns, total: 2.1 s
Wall time: 2.1 s


In [11]:
groups_df['Sucursal'].value_counts()

BLAA      24284
TUN        3165
IBA        2339
PAS        2089
POP        2078
NEI        1978
PER        1922
MAN        1159
GIR        1090
IPI        1041
CAR         758
SAN         728
CASAGO      686
VAL         660
FLO         611
ARM         575
HON         544
LET         526
BUC         511
QUI         478
STM         438
SIN         438
BUE         337
BAR         328
VIL         298
CAL         287
RIO         250
CUC         236
MED         219
MON         209
OTRO         10
Name: Sucursal, dtype: int64

In [12]:
groups_df['Sucursal'].nunique()

31

#### Ciudad

In [13]:
groups_df['Ciudad'].nunique()

524

In [14]:
groups_df['Ciudad'].value_counts()

bogota                                               23453
tunja                                                 2968
ibague                                                2305
popayan                                               2028
neiva                                                 1786
pasto                                                 1702
pereira                                               1478
manizales                                             1136
ipiales                                               1020
girardot                                               910
cartagena                                              731
san andres                                             679
valledupar                                             649
florencia                                              607
armenia                                                492
leticia                                                487
quibdo                                                 4

In [15]:
%%time

sucursales_ciudades = {
    'ARM': 'armenia',
    'BAR': 'barranquilla',
    'BLAA': 'bogota',
    'BUE': 'buenaventura',
    'BUC': 'bucaramanga',
    'CAL': 'cali',
    'CAR': 'cartagena',
    'CASAGO': 'bogota',
    'CATMAN': 'bogota',
    'CUC': 'cucuta',
    'FLO': 'florencia',
    'GIR': 'girardot',
    'HON': 'honda',
    'IBA': 'ibague',
    'IEC': 'bogota',
    'IPI': 'ipiales',
    'LET': 'leticia',
    'MAN': 'manizales',
    'MED': 'medellin',
    'MON': 'monteria',
    'NEI': 'neiva',
    'PAS': 'pasto',
    'PER': 'pereira',
    'POP': 'popayan',
    'QUI': 'quibdo',
    'RIO': 'riohacha',
    'SAN': 'san andres',
    'SIN': 'sincelejo',
    'STM': 'santa marta',
    'TUN': 'tunja',
    'VAL': 'valledupar',
    'VIL': 'villavicencio',
}

ciudades_ciudades = {
    'armenia': 'Armenia',
    'barranquilla': 'Barranquilla',
    'bogota': 'Bogotá',
    'bucaramanga': 'Bucaramanga',
    'buenaventura': 'Buenaventura',
    'cali': 'Cali',
    'cartagena': 'Cartagena',
    'cucuta': 'Cúcuta',
    'florencia': 'Florencia',
    'girardot': 'Girardot',
    'honda': 'Honda',
    'ibague': 'Ibagué',
    'ipiales': 'Ipiales',
    'leticia': 'Leticia',
    'manizales': 'Manizales',
    'medellin': 'Medellín',
    'monteria': 'Montería',
    'neiva': 'Neiva',
    'pasto': 'Pasto',
    'pereira': 'Pereira',
    'popayan': 'Popayán',
    'quibdo': 'Quibdó',
    'riohacha': 'Riohacha',
    'san andres': 'San Andrés',
    'santa marta': 'Santa Marta',
    'sincelejo': 'Sincelejo',
    'tunja': 'Tunja',
    'valledupar': 'Valledupar',
    'villavicencio': 'Villavicencio',
}

def fill_blank_ciudad(x):
    r = x['Ciudad']
    if not r:
        s = x['Sucursal']
        if s in sucursales_ciudades:
            r = sucursales_ciudades[s]
    return r

groups_df['Ciudad'] = groups_df.apply(fill_blank_ciudad, axis=1)

CPU times: user 1.36 s, sys: 6 µs, total: 1.36 s
Wall time: 1.35 s


In [16]:
%%time

dtypes = {
    'Ciudad': str,
    'Población': np.int32,
    'Departamento': str,
}

ciudades_df = pd.read_csv(
    'ciudades_colombia.csv',
    sep='\t',
    header=-1,
    names=['Wiki index', 'Ciudad', 'Población', 'Departamento'],
    usecols=['Ciudad', 'Población', 'Departamento'],
    dtype=dtypes,
    thousands='.',
)

def process_field_ciudad(x):
    r = x['Ciudad'].strip().lower()
    r = r.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    return r

ciudades_df['Ciudad'] = ciudades_df.apply(process_field_ciudad, axis=1)
cs = ciudades_df['Ciudad'].unique()

cn = groups_df['Ciudad'].value_counts()

def group_ciudad(x):
    e = x['Ciudad']
    if e and e in cs and cn[e] >= 10:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Ciudad'] = groups_df.apply(group_ciudad, axis=1)

CPU times: user 4.23 s, sys: 3.01 ms, total: 4.23 s
Wall time: 4.23 s


In [17]:
groups_df['Ciudad'].value_counts()

bogota           23496
tunja             2970
OTRO              2816
ibague            2309
popayan           2033
neiva             1787
pereira           1494
manizales         1140
ipiales           1020
girardot           910
cartagena          732
san andres         684
valledupar         651
florencia          607
armenia            495
leticia            491
quibdo             467
santa marta        436
sincelejo          428
bucaramanga        381
pasca              365
dosquebradas       362
honda              345
buenaventura       335
soacha             324
villavicencio      302
cali               273
barranquilla       250
riohacha           242
medellin           223
                 ...  
calarca             23
envigado            23
circasia            21
aipe                19
ricaurte            18
yaguara             17
tabio               17
sibate              17
paipa               16
villavieja          16
baraya              16
rivera              16
motavita   

In [18]:
groups_df['Ciudad'].nunique()

83

#### Tratamiento

In [19]:
groups_df['Tratamiento'].nunique()

5

In [20]:
groups_df['Tratamiento'].value_counts()

sr.          22505
sra.         22341
              4995
otra inf.      392
inst            39
Name: Tratamiento, dtype: int64

In [21]:
%%time

cn = groups_df['Tratamiento'].value_counts()

def group_tratamiento(x):
    e = x['Tratamiento']
    if e and (e == 'sr.' or e == 'sra.'):
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Tratamiento'] = groups_df.apply(group_tratamiento, axis=1)

CPU times: user 1.41 s, sys: 23 µs, total: 1.41 s
Wall time: 1.41 s


In [22]:
groups_df['Tratamiento'].value_counts()

sr.     22505
sra.    22341
OTRO     5426
Name: Tratamiento, dtype: int64

In [23]:
groups_df['Tratamiento'].nunique()

3

#### Barrio

In [24]:
groups_df['Barrio'].nunique()

9140

In [25]:
groups_df['Barrio'].value_counts()

centro                             1205
chapinero                           433
                                    365
candelaria                          294
palermo                             268
soledad                             253
castilla                            247
fontibon                            244
cedritos                            241
nieves                              239
kennedy                             227
modelia                             218
santa isabel                        211
recuerdo                            208
san luis                            201
americas                            196
santa barbara                       185
santa ines                          183
belen                               182
teusaquillo                         168
gaitan                              167
san antonio                         161
ciudad salitre                      160
obrero                              155
san francisco                       150


In [26]:
%%time

cn = groups_df['Barrio'].value_counts()

def group_barrio(x):
    e = x['Barrio']
    if e and cn[e] >= 2:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Barrio'] = groups_df.apply(group_barrio, axis=1)

CPU times: user 2.05 s, sys: 4 µs, total: 2.05 s
Wall time: 2.05 s


In [27]:
groups_df['Barrio'].value_counts()

OTRO                            6085
centro                          1205
chapinero                        433
candelaria                       294
palermo                          268
soledad                          253
castilla                         247
fontibon                         244
cedritos                         241
nieves                           239
kennedy                          227
modelia                          218
santa isabel                     211
recuerdo                         208
san luis                         201
americas                         196
santa barbara                    185
santa ines                       183
belen                            182
teusaquillo                      168
gaitan                           167
san antonio                      161
ciudad salitre                   160
obrero                           155
san francisco                    150
galerias                         145
centenario                       144
s

In [28]:
groups_df['Barrio'].nunique()

3420

#### CATEG

In [29]:
groups_df['CATEG'].nunique()

15

In [30]:
groups_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
Name: CATEG, dtype: int64

In [31]:
%%time

cn = groups_df['CATEG'].value_counts()

def group_categoria(x):
    e = x['CATEG']
    if e and cn[e] >= 0:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['CATEG'] = groups_df.apply(group_categoria, axis=1)

CPU times: user 2.01 s, sys: 3.92 ms, total: 2.02 s
Wall time: 2.01 s


In [32]:
groups_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
Name: CATEG, dtype: int64

In [33]:
groups_df['CATEG'].nunique()

15

#### Ocupación1

In [34]:
groups_df['Ocupación1'].nunique()

30

In [35]:
groups_df['Ocupación1'].value_counts()

estudiante                 35805
empleado                    3908
docente                     2946
independiente               2549
                            1799
trabajador y estudiante     1016
pensionado                   920
hogar                        507
investigador                 258
desempleado                  181
universidad                  142
colegios                      91
cultural                      33
salud                         30
comercio                      28
instuto técnico               15
agropecuario                  12
instuto tecnológico            5
editorial                      5
alimentos                      3
financiero                     3
congregación religiosa         3
transporte                     2
turismo                        2
plástico                       2
telecomunicaciones             2
construcción                   2
minero y energético            1
bebidas                        1
automotriz                     1
Name: Ocup

In [36]:
%%time

cn = groups_df['Ocupación1'].value_counts()

def group_ocupacion(x):
    e = x['Ocupación1']
    if e and cn[e] >= 10:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Ocupación1'] = groups_df.apply(group_ocupacion, axis=1)

CPU times: user 2 s, sys: 0 ns, total: 2 s
Wall time: 2 s


In [37]:
groups_df['Ocupación1'].value_counts()

estudiante                 35805
empleado                    3908
docente                     2946
independiente               2549
OTRO                        1831
trabajador y estudiante     1016
pensionado                   920
hogar                        507
investigador                 258
desempleado                  181
universidad                  142
colegios                      91
cultural                      33
salud                         30
comercio                      28
instuto técnico               15
agropecuario                  12
Name: Ocupación1, dtype: int64

In [38]:
groups_df['Ocupación1'].nunique()

17

#### Nivel Estudio1

In [39]:
groups_df['Nivel Estudio1'].nunique()

13

In [40]:
groups_df['Nivel Estudio1'].value_counts()

universitario 1 nivel    9657
universitario 2 nivel    8121
profesional              6882
secundaria               6875
básica primaria          6660
                         6375
maestría                 2180
postgrado                1345
técnico                   711
graduado                  486
doctorado                 437
tecnológico               335
ninguno                   208
Name: Nivel Estudio1, dtype: int64

In [41]:
%%time

cn = groups_df['Nivel Estudio1'].value_counts()

def group_nivel_estudio(x):
    e = x['Nivel Estudio1']
    if e and cn[e] >= 0:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Nivel Estudio1'] = groups_df.apply(group_nivel_estudio, axis=1)

CPU times: user 1.93 s, sys: 55 µs, total: 1.93 s
Wall time: 1.93 s


In [42]:
groups_df['Nivel Estudio1'].value_counts()

universitario 1 nivel    9657
universitario 2 nivel    8121
profesional              6882
secundaria               6875
básica primaria          6660
OTRO                     6375
maestría                 2180
postgrado                1345
técnico                   711
graduado                  486
doctorado                 437
tecnológico               335
ninguno                   208
Name: Nivel Estudio1, dtype: int64

In [43]:
groups_df['Nivel Estudio1'].nunique()

13

#### Área de Estudio1

In [44]:
groups_df['Área de Estudio1'].nunique()

172

In [45]:
groups_df['Área de Estudio1'].value_counts()

                                23191
derecho                          3448
literatura                       1508
pedagogía                        1151
ciencias sociales                1131
psicología                       1076
historia                         1056
filosofía y letras               1020
idiomas                           857
escolar                           841
economía                          810
sociología                        606
medicina                          602
antropología                      566
arquitectura                      550
ciencias políticas                538
biología                          499
artes plásticas                   468
contaduría                        432
artes escénicas                   430
ing. industrial                   411
adm. de empresas                  400
ing. de sistemas                  381
comunicación social               369
ing. civil                        353
música                            353
matemáticas 

In [46]:
%%time

cn = groups_df['Área de Estudio1'].value_counts()

def group_area_estudio(x):
    e = x['Área de Estudio1']
    if e and cn[e] >= 10:
        r = e
    else:
        r = 'OTRO'
    return r

groups_df['Área de Estudio1'] = groups_df.apply(group_area_estudio, axis=1)

CPU times: user 1.69 s, sys: 0 ns, total: 1.69 s
Wall time: 1.68 s


In [47]:
groups_df['Área de Estudio1'].value_counts()

OTRO                              23470
derecho                            3448
literatura                         1508
pedagogía                          1151
ciencias sociales                  1131
psicología                         1076
historia                           1056
filosofía y letras                 1020
idiomas                             857
escolar                             841
economía                            810
sociología                          606
medicina                            602
antropología                        566
arquitectura                        550
ciencias políticas                  538
biología                            499
artes plásticas                     468
contaduría                          432
artes escénicas                     430
ing. industrial                     411
adm. de empresas                    400
ing. de sistemas                    381
comunicación social                 369
música                              353


In [48]:
groups_df['Área de Estudio1'].nunique()

113

#### Edad

In [49]:
groups_df['Edad'].nunique()

100

In [50]:
groups_df['Edad'].value_counts()

        6626
22.0    1928
21.0    1860
23.0    1819
20.0    1770
24.0    1653
19.0    1539
17.0    1539
18.0    1463
25.0    1459
16.0    1320
26.0    1235
27.0    1124
28.0    1068
10.0    1001
9.0      993
15.0     989
11.0     960
30.0     934
29.0     927
8.0      887
12.0     817
32.0     794
31.0     793
13.0     766
33.0     696
7.0      694
14.0     692
35.0     677
34.0     647
        ... 
69.0      94
71.0      86
70.0      84
72.0      79
74.0      58
73.0      53
76.0      42
75.0      40
78.0      32
77.0      29
81.0      14
80.0      11
84.0      11
79.0      10
83.0       9
82.0       8
86.0       7
89.0       5
88.0       4
87.0       3
93.0       2
85.0       2
90.0       1
91.0       1
92.0       1
94.0       1
95.0       1
97.0       1
98.0       1
99.0       1
Name: Edad, dtype: int64

In [51]:
%%time

def fill_blank_edad(x):
    r = x['Edad']
    if not r:
        r = -1
    else:
        r = int(r)
    return r

groups_df['Edad'] = groups_df.apply(fill_blank_edad, axis=1)

CPU times: user 1.49 s, sys: 0 ns, total: 1.49 s
Wall time: 1.49 s


In [52]:
groups_df['Edad'].value_counts()

-1     6626
 22    1928
 21    1860
 23    1819
 20    1770
 24    1653
 17    1539
 19    1539
 18    1463
 25    1459
 16    1320
 26    1235
 27    1124
 28    1068
 10    1001
 9      993
 15     989
 11     960
 30     934
 29     927
 8      887
 12     817
 32     794
 31     793
 13     766
 33     696
 7      694
 14     692
 35     677
 34     647
       ... 
 69      94
 71      86
 70      84
 72      79
 74      58
 73      53
 76      42
 75      40
 78      32
 77      29
 81      14
 84      11
 80      11
 79      10
 83       9
 82       8
 86       7
 89       5
 88       4
 87       3
 93       2
 85       2
 92       1
 98       1
 99       1
 91       1
 97       1
 90       1
 94       1
 95       1
Name: Edad, dtype: int64

In [53]:
groups_df['Edad'].nunique()

100

#### Rango_edad

In [54]:
%%time

def bins_to_labels(bins):
    labels = []
    labels.append('OTRO'.format(bins[1]))
    labels.append('{}-'.format(bins[2]))
    for i in range(2, len(bins) - 2):
        labels.append('{}-{}'.format(bins[i]+1, bins[i+1]))
    labels.append('{}+'.format(bins[-2]+1))
    return labels

# bins = [-2, -1, 11, 17, 25, 65, 100]
# bins = [-2, -1, 15, 30, 40, 50, 64, 999]
bins = [-2, -1, 17, 29, 39, 49, 64, 999]
labels = bins_to_labels(bins)

def calculate_rango(x):
    pd.cut(pd.Series(x), bins=bins, labels=labels)

groups_df['Rango_edad'] = pd.cut(groups_df['Edad'], bins=bins, labels=labels)

CPU times: user 2.72 ms, sys: 0 ns, total: 2.72 ms
Wall time: 2.57 ms


In [55]:
groups_df['Rango_edad'].value_counts()

18-29    17845
17-      12770
OTRO      6626
30-39     6609
40-49     2617
50-64     2541
65+       1264
Name: Rango_edad, dtype: int64

In [56]:
groups_df['Rango_edad'].nunique()

7

## Resultados finales

In [57]:
groups_df.shape

(50272, 12)

In [58]:
groups_df.to_csv('usuarios_2016.csv', index=False)